In [ ]:
import pandas as pd
from astropy.table import Table
from ctapipe.instrument import SubarrayDescription
from lstchain.io.io import dl1_params_lstcam_key, dl1_images_lstcam_key
from lstchain.io import read_calibration_file
import sys
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
from astropy.io import ascii
import numpy as np
import pandas as pd
import matplotlib
import csv
import scipy
import matplotlib.pyplot as plt
from seaborn import histplot
import random
import copy
from os import path, makedirs
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from ctapipe.containers import MuonEfficiencyContainer
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from ctapipe.io import EventSource, EventSeeker
from ctapipe.core import TelescopeComponent
from ctapipe.core.traits import (
    List,
    Int,
    FloatTelescopeParameter,
    TelescopeParameter,
    Unicode,
)
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
from traitlets.config.loader import Config, FileConfigLoader, JSONFileConfigLoader
import pathlib
from calibpipe.throughput import MuonCalibrator
from traitlets.config import Config
from astropy.time import Time
from astropy.coordinates import EarthLocation,SkyCoord, AltAz 
from traitlets.config import Config
import time
from ctapipe.image import number_of_islands
import json
from pyirf import spectral
from lstchain.image.muon import analyze_muon_event, tag_pix_thr





In [ ]:


filename = '/Users/vdk/dl1_simtel_corsika_run32.h5'
subarray = SubarrayDescription.from_hdf(filename)

images = Table.read(filename, path=dl1_images_lstcam_key)['image']
parameters = pd.read_hdf(filename, key=dl1_params_lstcam_key)

pd.set_option('display.max_columns', None)
for i in parameters['tel_id'][:]:
    if i != 1:
        print(f"Other telescope = {i}")

In [ ]:
parameters

### CTApipe Observation Processing


In [ ]:
# Треба завантажити конфіг в івент соурс


config_filename = '/Users/vdk/RealLST/typical_run_data/lstchain_standard_v0.10_heuristic_ff.json'

with open(config_filename) as json_file:
    config_data = json.load(json_file)

custom_config = Config(config_data)

muon_processor_config = Config({
                                "MuonProcessor": {
                                    "RingQuery": {
                                        "quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.1"],
                                                        ["ring_completeness", "parameters.completeness > 0.1"]]},
                                    "ImageParameterQuery" :{
                                        "quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

cfg = Config({
    "source_config": {
        "EventSource" : {
            "allowed_tels": [1],"max_events": 9999}},
        "PointingSource":{
            "drive_report_path": '/Users/vdk/RealLST/data/DrivePosition_log_20231007.txt'},
        "LSTR0Corrections": {
          "calib_scale_high_gain":1.088,
          "calib_scale_low_gain":1.004,
          "drs4_pedestal_path": '/Users/vdk/RealLST/data/drs4_pedestal.Run14937.0000.h5',
          "calibration_path": '/Users/vdk/RealLST/data/calibration_filters_52.Run14938.0000.h5',
          "drs4_time_calibration_path": '/Users/vdk/RealLST/data/time_calibration.Run08349.0000.h5'
      }})

config = replace_config(standard_config, muon_processor_config)
#onfig = replace_config(config, muon_processor_config_2)
config = replace_config(config, custom_config)

image_processor = ImageProcessor(source.subarray)
muon_processor = MuonProcessor(source.subarray, config = Config(config))
calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray, config = Config(config))


### CTApipe 4LST processing

In [ ]:
int_cut = 0.2
min_pixel = 70

muon_processor_config = Config({
                                "MuonProcessor": {
                                    "RingQuery": {
                                        "quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.1"],
                                                        ["ring_completeness", "parameters.completeness > 0.1"]]},
                                    "ImageParameterQuery" :{
                                        "quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

In [ ]:
event_ids_calib = []
impacts = []
widths = []
for k in range(1,101):
    filename = f'/Users/vdk/LST/4LSTmuons/simtel_run{k}01_muon.simtel.gz'
    source = EventSource(filename)
    event_iterator = iter(source)
    for i,event in enumerate(event_iterator):
        for m in event.trigger.tels_with_trigger:
            checker += 1
            image_processor = ImageProcessor(source.subarray)
            muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
            calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
            calib(event)
            image_processor(event)
            muon_processor(event)
            for m in event.muon.tel.keys():
                if not np.isnan(event.muon.tel[m].efficiency.width) and (0.2*11 < event.muon.tel[m].efficiency.impact.to_value() < 0.9*11):
                    event_container[keks+i] = event
                    event_ids_calib.append(event.index.event_id)
                    impacts.append(event.muon.tel[m].efficiency.impact)
                    widths.append(event.muon.tel[m].efficiency.width)
    t2 = time.perf_counter(), time.process_time()
    print("checker = ", checker)   

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")

In [ ]:
print("muons = ", len(event_container))
print("muons events = ", event_container.keys())

opt_eff_m = []
for i in event_container:
    for m in event_container[i].trigger.tels_with_trigger:
        if not np.isnan(event_container[i].muon.tel[m].efficiency.optical_efficiency):
            opt_eff_m.append(event_container[i].muon.tel[m].efficiency.optical_efficiency)

print("mean = ", np.mean(opt_eff_m))
print("std = ", np.std(opt_eff_m))
print("len = ", len(opt_eff_m))
print(f"event_ids = {event_ids_calib}")


In [ ]:
plt.hist(widths, bins = 12, alpha = 0.7)
plt.ylabel('Counts')
plt.xlabel('Ring widths in deg')
plt.xlim(0,0.12)
plt.ylim(0,90)
plt.grid(alpha = 0.5)
plt.title("Widths of muons processed by CalibPipe method")

In [ ]:
plt.hist(impacts, bins = 10, alpha = 0.7)
plt.ylabel('Counts')
plt.xlabel('Ring impact in m')
plt.xlim(0,12)
plt.ylim(0,90)
plt.grid(alpha = 0.5)
plt.title("Impact of muons processed by CalibPipe method")

### lstchain processing

In [ ]:


int_cut = 0.3
min_pixel = 70

muon_processor_config = Config({
                                "MuonProcessor": {
                                    "RingQuery": {
                                        "quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.1"],
                                                        ["ring_completeness", "parameters.completeness > 0.1"]]},
                                    "ImageParameterQuery" : {
                                        "quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]},
                                    "MuonIntensityFitter" : {
                                        "hole_radius_m" : [["type", "LST_*", 0.308]]
                                    }}})

checker = 0
event_container = {}

muon_params = []
event_ids_lstchain = []

for k in range(1,101):
    filename = f'/Users/vdk/LST/4LSTmuons/simtel_run{k}01_muon.simtel.gz'
    print(filename)
    source = EventSource(filename)
    event_iterator = iter(source)

    for i,event in enumerate(event_iterator):
        for m in event.trigger.tels_with_trigger:
            checker += 1
            image_processor = ImageProcessor(source.subarray)
            muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
            #muon_processor = MuonProcessor(source.subarray)
            calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
            #print(f"Non Muon Event Telescope with stereo trigger = {event.trigger.tels_with_trigger}")
            calib(event)
            #image_processor(event)
            #muon_processor(event)
            event_id = event.index.event_id
            for tel_id in event.trigger.tels_with_trigger:
                image = event.dl1.tel[tel_id].image
                if not tag_pix_thr(event.dl1.tel[tel_id].image):  # default skips pedestal and calibration events
                    continue

                (
                    muonintensityparam, dist_mask, size, size_outside_ring,
                    muonringparam, good_ring, radial_distribution,
                    mean_pixel_charge_around_ring, muonparameters
                ) = analyze_muon_event(source.subarray, tel_id, event_id,
                                    image, good_ring_config=None,
                                    plot_rings=False, plots_path=False
                                    )
                if good_ring:
                    muon_params.append(muonintensityparam)
                    event_ids_lstchain.append(event.index.event_id)


In [ ]:
event.pointing.tel[1]

In [ ]:
opt_eff = []
for muon_container in muon_params:
    opt_eff.append(muon_container.optical_efficiency)
    #print(muon_container.optical_efficiency)

In [ ]:
print(f"mean = {np.mean(opt_eff)}")
print(f"len =  {len(opt_eff)}")

In [ ]:
source.subarray.tels[1].camera.geometry

In [ ]:
widths = []
impacts = []
for muon_container in muon_params:
    widths.append(muon_container.width)
    impacts.append(muon_container.impact)

In [ ]:
plt.hist(widths, bins = 12, alpha = 0.7)
plt.ylabel('Counts')
plt.xlabel('Ring widths in deg')
plt.xlim(0,0.12)
plt.ylim(0,90)
plt.grid(alpha = 0.5)
plt.title("Widths of muons processed by lstchain method")

In [ ]:
plt.hist(impacts, bins = 10, alpha = 0.7)
plt.ylabel('Counts')
plt.xlabel('Ring impact in m')
plt.xlim(0,12)
plt.ylim(0,90)
plt.grid(alpha = 0.5)
plt.title("Impact of muons processed by lstchain method")

In [ ]:
muon_params

### Modified ctapipe

In [ ]:
int_cut = 0.1
min_pixel = 50

muon_processor_config = Config({
                                "MuonProcessor": {
                                    "RingQuery": {
                                        "quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.3"],
                                                        ["ring_completeness", "parameters.completeness > 0.3"]]},
                                    "ImageParameterQuery" : {
                                        "quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]},
                                    "MuonIntensityFitter" : {
                                        "hole_radius_m" : [["type", "LST_*", 0.308]]
                                    }}})

In [ ]:
from ctapipe.core.traits import Bool, BoolTelescopeParameter, ComponentName
from ctapipe.image.cleaning import ImageCleaner


#event_ids_calib = []
impacts = []
widths = []

image_cleaner_type = ComponentName(
        ImageCleaner, default_value="TailcutsImageCleaner"
    )
for k in range(1,11):
    filename = f'/Users/vdk/LST/4LSTmuons/simtel_run{k}01_muon.simtel.gz'
    source = EventSource(filename)
    event_iterator = iter(source)
    geom = source.subarray.tels[1].camera.geometry
    for i,event in enumerate(event_iterator):
        for m in event.trigger.tels_with_trigger:
            #checker += 1
            image_processor = ImageProcessor(source.subarray)
            muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)

            #clean = ImageCleaner.from_name(image_cleaner_type, subarray=source.subarray)

            #calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
            calib = CameraCalibrator(image_extractor_type="LocalPeakWindowSum",subarray = source.subarray)
            calib(event)
            image_processor(event)
            
            #for tel_id, dl1_camera in event.dl1.tel.items():
            #        dl1_camera.image_mask = tailcuts_clean(geom, image,picture_thresh=10,boundary_thresh=5)
                #dl1_camera.image_mask = self.clean(tel_id=tel_id,image=dl1_camera.image,arrival_times=dl1_camera.peak_time)
            
            muon_processor(event)
            for m in event.muon.tel.keys():
                if not np.isnan(event.muon.tel[m].efficiency.width) and (0.2*11 < event.muon.tel[m].efficiency.impact.to_value() < 0.9*11):
                    event_container[keks+i] = event
                    #event_ids_calib.append(event.index.event_id)
                    impacts.append(event.muon.tel[m].efficiency.impact)
                    widths.append(event.muon.tel[m].efficiency.width)
    t2 = time.perf_counter(), time.process_time()
    print("checker = ", checker)   

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")


In [ ]:
opt_eff_m = []
for i in event_container:
    for m in event_container[i].trigger.tels_with_trigger:
        if not np.isnan(event_container[i].muon.tel[m].efficiency.optical_efficiency):
            opt_eff_m.append(event_container[i].muon.tel[m].efficiency.optical_efficiency)

print("mean = ", np.mean(opt_eff_m))
print("std = ", np.std(opt_eff_m))
print("len = ", len(opt_eff_m))
print(f"event_ids = {event_ids_calib}")

### Event id processing

In [ ]:
event_ids_calib

In [ ]:
common_events = []
for event_id in event_ids_calib:
    if event_id in event_ids_lstchain:
        common_events.append(event_id)


In [ ]:
checker = 0

lstchain_container = {}

for k in range(1,41):
    filename = f'/Users/vdk/LST/4LSTmuons/simtel_run{k}01_muon.simtel.gz'
    print(filename)
    source = EventSource(filename)
    event_iterator = iter(source)

    for i,event in enumerate(event_iterator):
        if event.index.event_id in common_events:
            for m in event.trigger.tels_with_trigger:
    
                muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
                calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
                
                calib(event)
                event_id = event.index.event_id
                for tel_id in event.trigger.tels_with_trigger:
                    image = event.dl1.tel[tel_id].image
                    if not tag_pix_thr(event.dl1.tel[tel_id].image):  # default skips pedestal and calibration events
                        continue

                    (
                        muonintensityparam, dist_mask, size, size_outside_ring,
                        muonringparam, good_ring, radial_distribution,
                        mean_pixel_charge_around_ring, muonparameters
                    ) = analyze_muon_event(source.subarray, tel_id, event_id,
                                        image, good_ring_config=None,
                                        plot_rings=False, plots_path=False
                                        )
            event.muon.tel[m].efficiency = muonintensityparam
            if checker < 10:
                lstchain_container[event.index.event_id] = event
                checker += 1 
            if checker == 10:
                break
                    

In [ ]:
calib_container = {}
checker = 0
for k in range(1,2):
    filename = f'/Users/vdk/LST/4LSTmuons/simtel_run{k}01_muon.simtel.gz'
    source = EventSource(filename)
    event_iterator = iter(source)
    for i,event in enumerate(event_iterator):
        if event.index.event_id in common_events: 
            for m in event.trigger.tels_with_trigger:
                image_processor = ImageProcessor(source.subarray)
                muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
                calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
                calib(event)
                #image_processor(event)
                muon_processor(event)

                if checker < 10:
                    calib_container[event.index.event_id] = event
                    checker += 1 
                if checker == 10:
                    break



In [ ]:
plot_stop = 0
for key in calib_container:
    for tels in calib_container[key].muon.tel.keys():
            plt.figure(figsize = (6,4))
            camgeom = source.subarray.tel[tels].camera.geometry
            title = f"event_id = {lstchain_container[key].index.event_id}, optical efficiency {round(calib_container[key].muon.tel[tels].efficiency.optical_efficiency,3)}"
            disp = CameraDisplay(camgeom,title=title)
            #disp.image = calib_container[key].simulation.tel[tels].true_image
            #disp.image = event_container[i].r1.tel[m].waveform.sum(axis=1)
            #print(np.shape(calib_container[key].dl1.tel[m].image))
            disp.image = calib_container[key].dl1.tel[tels].image
            disp.cmap = plt.cm.RdBu_r
            disp.add_colorbar()
            disp.set_limits_percent(95)
            plt.show()
            #plt.savefig(f"/Users/vdk/3978-4000runs_MuonsCleaning+Island17bin_intensitycut<0.3_event{i}.png")
    plot_stop += 1
    if plot_stop >2:
        break
    

In [ ]:
plot_stop = 0
for key in lstchain_container:
    for tels in lstchain_container[key].trigger.tels_with_trigger:
            plt.figure(figsize = (6,4))
            camgeom = source.subarray.tel[tels].camera.geometry
            title = f"event_id = {lstchain_container[key].index.event_id}, optical efficiency {round(lstchain_container[key].muon.tel[tels].efficiency.optical_efficiency,3)}"
            disp = CameraDisplay(camgeom,title=title)
            #disp.image = lstchain_container[key].simulation.tel[tels].true_image
            #disp.image = event_container[i].r1.tel[m].waveform.sum(axis=1)
            disp.image = lstchain_container[key].dl1.tel[tels].image
            disp.cmap = plt.cm.RdBu_r
            disp.add_colorbar()
            disp.set_limits_percent(95)
            plt.show()
            #plt.savefig(f"/Users/vdk/3978-4000runs_MuonsCleaning+Island17bin_intensitycut<0.3_event{i}.png")
    plot_stop += 1
    if plot_stop > 2: 
        break

In [ ]:
print(lstchain_container[48900].dl1.tel[2])

In [ ]:
print(calib_container[48900].dl1.tel[2].image)

In [ ]:
len(event.trigger.tels_with_trigger)

#### Check dl1 images and peak time for different methods

In [ ]:
#event_id = 213300
event_id = 48900
tel = calib_container[event_id].trigger.tels_with_trigger[0]
np.count_nonzero(calib_container[event_id].dl1.tel[tel].image == lstchain_container[event_id].dl1.tel[tel].image)

In [ ]:
np.count_nonzero(calib_container[event_id].dl1.tel[tel].peak_time == lstchain_container[event_id].dl1.tel[tel].peak_time)

In [ ]:
print(lstchain_container[event_id].muon.tel[tel])

In [ ]:
print(calib_container[event_id].muon.tel[tel])

In [ ]:
event

### Check intensity fitter parameters

In [ ]:


int_cut = 0.3
min_pixel = 70

muon_processor_config = Config({
                                "MuonProcessor": {
                                    "RingQuery": {
                                        "quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.1"],
                                                        ["ring_completeness", "parameters.completeness > 0.1"]]},
                                    "ImageParameterQuery" : {
                                        "quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]},
                                    "MuonIntensityFitter" : {
                                        "hole_radius_m" : [["type", "LST_*", 0.308]]
                                    }}})

checker = 0
event_container = {}

muon_params = []
event_ids_lstchain = []

for k in range(1,2):
    filename = f'/Users/vdk/LST/4LSTmuons/simtel_run{k}01_muon.simtel.gz'
    print(filename)
    source = EventSource(filename)
    event_iterator = iter(source)

    for i,event in enumerate(event_iterator):
        if event.index.event_id in [48900, 213300, 1154900]:
            checker += 1
            for m in event.trigger.tels_with_trigger:
                checker += 1
                image_processor = ImageProcessor(source.subarray)
                muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
                #muon_processor = MuonProcessor(source.subarray)
                calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
                #print(f"Non Muon Event Telescope with stereo trigger = {event.trigger.tels_with_trigger}")
                calib(event)
                #image_processor(event)
                #muon_processor(event)
                event_id = event.index.event_id
                for tel_id in event.trigger.tels_with_trigger:
                    image = event.dl1.tel[tel_id].image
                    if not tag_pix_thr(event.dl1.tel[tel_id].image):  # default skips pedestal and calibration events
                        continue

                    (
                        muonintensityparam, dist_mask, size, size_outside_ring,
                        muonringparam, good_ring, radial_distribution,
                        mean_pixel_charge_around_ring, muonparameters
                    ) = analyze_muon_event(source.subarray, tel_id, event_id,
                                        image, good_ring_config=None,
                                        plot_rings=False, plots_path=False
                                        )
                    if good_ring:
                        muon_params.append(muonintensityparam)
                        event_ids_lstchain.append(event.index.event_id)
                    break


In [ ]:
print(muon_params[2])

In [ ]:
int_cut = 1
min_pixel = 50

muon_processor_config = Config({
                                "MuonProcessor": {
                                    "RingQuery": {
                                        "quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.1"],
                                                        ["ring_completeness", "parameters.completeness > 0.1"]]},
                                    "ImageParameterQuery" : {
                                        "quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]},
                                    "MuonIntensityFitter" : {
                                        "hole_radius_m" : [["type", "LST_*", 0.308]]
                                    }}})


event_ids_calib = []
impacts = []
widths = []
for k in range(1,2):
    filename = f'/Users/vdk/LST/4LSTmuons/simtel_run{k}01_muon.simtel.gz'
    source = EventSource(filename)
    event_iterator = iter(source)
    for i,event in enumerate(event_iterator):
        if event.index.event_id in [48900, 213300, 1154900]:
            for m in event.trigger.tels_with_trigger:
                
                image_processor = ImageProcessor(source.subarray)
                muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
                calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
                calib(event)
                image_processor(event)
                muon_processor(event)



In [ ]:
telescope_description = source.subarray.tel[1]

In [ ]:
telescope_description.optics
optics = telescope_description.optics
mirror_area = optics.mirror_area.to_value(u.m**2)
mirror_radius = np.sqrt(mirror_area / np.pi)

focal_length = optics.equivalent_focal_length

In [ ]:
cam = telescope_description.camera.geometry
camera_frame = CameraFrame(focal_length=focal_length, rotation=cam.cam_rotation)
cam_coords = SkyCoord(x=cam.pix_x, y=cam.pix_y, frame=camera_frame)
tel_coords = cam_coords.transform_to(TelescopeFrame())

In [ ]:
pixel_diameter = 2 * (np.sqrt(cam.pix_area[0] / np.pi) / focal_length * u.rad).to_value(u.rad)

pixel_diameter


### Look into image and mask

In [ ]:
np.set_printoptions(threshold=sys.maxsize)
ctapipe_1154900_mask = np.loadtxt('/Users/vdk/ctapipe_1154900_mask.txt')
ctapipe_1154900_image = np.loadtxt('/Users/vdk/ctapipe_1154900_image.txt')
lstchain_1154900_mask = np.loadtxt('/Users/vdk/lstchain_1154900_mask.txt')
lstchain_1154900_image = np.loadtxt('/Users/vdk/lstchain_1154900_image.txt')

ctapipe_213300_mask = np.loadtxt('/Users/vdk/ctapipe_213300_mask.txt')
ctapipe_213300_image = np.loadtxt('/Users/vdk/ctapipe_213300_image.txt')
lstchain_213300_mask = np.loadtxt('/Users/vdk/lstchain_213300_mask.txt')
lstchain_213300_image = np.loadtxt('/Users/vdk/lstchain_213300_image.txt')

In [ ]:
for i,j in enumerate(ctapipe_213300_image):
    #if j != lstchain_213300_image[i]:
    if i in [42,188,1600]:
        print(f"ctapipe = {j} -------- lstchain = {lstchain_213300_image[i]}")

In [ ]:
counter = 0
for i,j in enumerate(ctapipe_213300_mask):
    if j != lstchain_213300_mask[i]:
        counter += 1
        print(f"ctapipe = {j} -------- lstchain = {lstchain_213300_mask[i]} for the {i}th pixel")

In [ ]:
counter

In [ ]:
np.count_nonzero(ctapipe_1154900_mask)
np.count_nonzero(lstchain_1154900_mask)

### Look into fov_lon fov_lat

In [ ]:
np.set_printoptions(threshold=sys.maxsize)
ctapipe_fovlon = np.loadtxt('/Users/vdk/ctapipe_48900_fovlon.txt')
ctapipe_fovlon_2 = np.loadtxt('/Users/vdk/ctapipe_213300_fovlon.txt')
ctapipe_fovlon_3 = np.loadtxt('/Users/vdk/ctapipe_1154900_fovlon.txt')
ctapipe_fovlat = np.loadtxt('/Users/vdk/ctapipe_48900_fovlat.txt')
lstchain_fovlat = np.loadtxt('/Users/vdk/lstchain-1.0669492483139038_fovlat.txt')
lstchain_fovlon_1 = np.loadtxt('/Users/vdk/lstchain-1.0669492483139038_fovlon.txt')
lstchain_fovlon_2 = np.loadtxt('/Users/vdk/lstchain1.7702226638793945_fovlon.txt')
lstchain_fovlon_3 = np.loadtxt('/Users/vdk/lstchain-1.2239160537719727_fovlon.txt')


ctapipe_fovlon_chng = np.loadtxt('/Users/vdk/ctapipe_48900_fovlon_changed.txt')


In [ ]:
counter = 0
for i,j in enumerate(lstchain_fovlon_1):
    if j != lstchain_fovlon_3[i]:
        counter += 1
        print(f"ctapipe = {j} -------- lstchain = {lstchain_fovlon_3[i]} for the {i}th pixel")

print(counter)

In [ ]:
counter = 0
for i,j in enumerate(ctapipe_fovlon):
    if j != ctapipe_fovlon_3[i]:
        counter += 1
        print(f"ctapipe = {j} -------- lstchain = {ctapipe_fovlon_3[i]} for the {i}th pixel")

print(counter)

In [ ]:
counter = 0
for i,j in enumerate(ctapipe_fovlon_chng):
    if j != lstchain_fovlon_1[i]:
        counter += 1
        print(f"ctapipe = {j} -------- lstchain = {lstchain_fovlon_1[i]} for the {i}th pixel")

print(counter)

In [ ]:
counter = 0
for i,j in enumerate(ctapipe_fovlat):
    if j != lstchain_fovlat[i]:
        counter += 1
        print(f"ctapipe = {j} -------- lstchain = {lstchain_fovlat[i]} for the {i}th pixel")

print(counter)

### Exploring geometry

#### User Guide ctapipe

In [ ]:
geometry = source.subarray.tel[tel_id].camera.geometry
pix_x = geometry.pix_x
pix_y = geometry.pix_y
focal_length = source.subarray.tel[tel_id].optics.equivalent_focal_length


camera_frame = CameraFrame(
focal_length=focal_length,
rotation=0 * u.deg,
)

cam_coords = SkyCoord(x=pix_x, y=pix_y, frame=camera_frame)

print(cam_coords)

#
# Pointing corrections should applied to the transformation between this frame and the camera frame. ??
#

telescope_frame = TelescopeFrame()

telescope_coords = cam_coords.transform_to(telescope_frame)

print(telescope_coords)


#### ctapipe realisation

In [ ]:
from ctapipe.coordinates import TelescopeFrame


tel_id = 2


frame = TelescopeFrame()

#geometries = {tel_id: tel.camera.geometry.transform_to(frame) for tel_id, tel in source.subarray.tel.items()}
#geometry = geometries[tel_id]


geometry = source.subarray.tel[tel_id].camera.geometry.transform_to(frame)



fov_lon = geometry.pix_x
fov_lat = geometry.pix_y


print(geometry.pix_x)

In [ ]:
# type(tel.camera.geometry) = <class 'ctapipe.instrument.camera.geometry.CameraGeometry'>
for tel_id, tel in source.subarray.tel.items():
    print(f"tel_id {tel_id} and tel {tel} and tel.camera.geometry {type(tel.camera.geometry)}")

In [ ]:
print(f"ctapipe realisation = {source.subarray.tel[tel_id].camera.geometry}")
print(f"lstchain realisation = {tel_description.camera.geometry}")

In [ ]:
source.subarray.tel[tel_id].optics.equivalent_focal_length

In [ ]:
geom

In [ ]:
print(geometries[2].pix_y)

#### lstchain realisation

In [ ]:
def pixel_coords_to_telescope(geom, equivalent_focal_length):
    """
    Get the x, y coordinates of the pixels in the telescope frame

    Parameters
    ----------
    geom : `CameraGeometry`
        Camera geometry
    equivalent_focal_length: `float`
        Focal length of the telescope

    Returns
    -------
    fov_lon, fov_lat : `floats`
        Coordinates in  the TelescopeFrame
    """

    camera_coord = SkyCoord(geom.pix_x, geom.pix_y,
                            CameraFrame(focal_length=equivalent_focal_length,
                                        rotation=geom.cam_rotation))

    tel_coord = camera_coord.transform_to(TelescopeFrame())

    return tel_coord.fov_lon, tel_coord.fov_lat


tel_description = source.subarray.tels[tel_id]

geom = tel_description.camera.geometry

equivalent_focal_length = tel_description.optics.equivalent_focal_length

x, y = pixel_coords_to_telescope(geom, equivalent_focal_length)

In [ ]:
tel_description.optics.effective_focal_length

In [ ]:
counter = 0
for i,j in enumerate(fov_lon):
    if j != x[i]:
        counter += 1
        print(f"ctapipe = {j} -------- lstchain = {x[i]} for the {i}th pixel")

print(counter)

### Other theories

#### Process only for LST1

In [ ]:
int_cut = 0.3
min_pixel = 70

muon_processor_config = Config({
                                "MuonProcessor": {
                                    "RingQuery": {
                                        "quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.1"],
                                                        ["ring_completeness", "parameters.completeness > 0.1"]]},
                                    "ImageParameterQuery" : {
                                        "quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]},
                                    "MuonIntensityFitter" : {
                                        "hole_radius_m" : [["type", "LST_*", 0.308]]
                                    }}})

checker = 0
event_container = {}

muon_params_lst1 = []
event_ids_lstchain_lst1 = []

for k in range(1,101):
    filename = f'/Users/vdk/LST/4LSTmuons/simtel_run{k}01_muon.simtel.gz'
    print(filename)
    source = EventSource(filename)
    event_iterator = iter(source)

    for i,event in enumerate(event_iterator):
        for m in event.trigger.tels_with_trigger:
            if m == 1:
                checker += 1
                image_processor = ImageProcessor(source.subarray)
                muon_processor = MuonProcessor(source.subarray, config = muon_processor_config)
                #muon_processor = MuonProcessor(source.subarray)
                calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
                #print(f"Non Muon Event Telescope with stereo trigger = {event.trigger.tels_with_trigger}")
                calib(event)
                #image_processor(event)
                #muon_processor(event)
                event_id = event.index.event_id
                tel_id = 1
                image = event.dl1.tel[tel_id].image
                if not tag_pix_thr(event.dl1.tel[tel_id].image):  # default skips pedestal and calibration events
                    continue

                (
                    muonintensityparam, dist_mask, size, size_outside_ring,
                    muonringparam, good_ring, radial_distribution,
                    mean_pixel_charge_around_ring, muonparameters
                ) = analyze_muon_event(source.subarray, tel_id, event_id,
                                    image, good_ring_config=None,
                                    plot_rings=False, plots_path=False
                                    )
                if good_ring:
                    muon_params_lst1.append(muonintensityparam)
                    event_ids_lstchain_lst1.append(event.index.event_id)


In [ ]:
#from datetime import datetime
timestamp_fmt=Unicode(        default_value="%Y-%m-%d %H:%M:%S",        help="Timestamp's format.",    )

In [ ]:
timestamp = event.trigger.tel[1].time

In [ ]:
new_timestamp = datetime.strptime(timestamp,timestamp_fmt)

In [ ]:
timestamp.value

In [ ]:
from astropy.time import Time

# Assuming your timestamp in TAI with format 'MJD' is stored in a variable called 'tai_mjd_timestamp'
tai_mjd_timestamp = 58984#event.trigger.tel[1].time  # Replace this with your TAI MJD timestamp



# Convert Astropy Time object to Python datetime object
datetime_obj = Time(tai_mjd_timestamp, format='mjd', scale='tai').to_datetime()

datetime_obj

In [ ]:
type(list(event.muon.tel.keys()))

In [ ]:
test = event.trigger.tels_with_trigger.copy()

In [ ]:
test
new_arr = []

In [ ]:
event.trigger.tels_with_trigger

In [ ]:
new_arr.append(test)

In [ ]:
new_arr.append(event.trigger.tels_with_trigger)

In [ ]:
test2 = np.append(test,4)

In [ ]:
new_arr.append(test2)

In [ ]:
lok = ','.join(map(str, new_arr))
lok

In [ ]:
datetime_obj = Time(event.trigger.tel[1].time, format='mjd', scale='tai').to_datetime()

In [ ]:
type(event.trigger.tel[1].time)

In [ ]:
tai_mjd_timestamp = 59353.5  # Replace this with your TAI MJD timestamp

# Create an Astropy Time object from the TAI MJD timestamp
t = Time(58353.5, format='mjd', scale='tai')

# Convert Astropy Time object to Python datetime object
datetime_obj_1 = t.to_datetime()

t = Time(59353.5, format='mjd', scale='tai')
datetime_obj_2 = t.to_datetime()

t = Time(57353.5, format='mjd', scale='tai')
datetime_obj_3 = t.to_datetime()



In [ ]:
datetime_obj_1

In [ ]:
datetime_obj_2

In [ ]:
list_times = [datetime_obj_1, datetime_obj_2, datetime_obj_3]

In [ ]:
type(min(list_times))

In [ ]:
timestamp_2 = 59353.5  # Replace this with your TAI MJD timestamp


timestamp_2 = Time(timestamp_2, format='mjd', scale='tai').to_datetime()



In [ ]:
type(timestamp_2)

In [ ]:
from datetime import datetime 
timestamp=Unicode(default_value=None,        help="The UTC timestamp of the e.g. relevant OB, "        "defining the timeseries to be retrieved from a data assimilation system.",        allow_none=False,)


timestamp_fmt=Unicode(default_value="%Y-%m-%d %H:%M:%S", help="Timestamp's format.",)


timestamp = datetime.strptime('1993-11-3 00:00:00', "%Y-%m-%d %H:%M:%S")

In [ ]:
type(timestamp)

In [ ]:
datetime = event.trigger.tel[1].time.to_datetime()

In [ ]:
datetime

In [ ]:
print(event.mon.tel[2].calibration.unusable_pixels)

In [ ]:
not np.isnan(event.muon.tel[3].efficiency.optical_efficiency)

In [ ]:
event.simulation.shower.energy.to_value()

In [ ]:
test = [1,2,3,4,5,6,7]
number = 100
output_energy = '/Users/vdk/energy_stereo.txt'
with open(output_energy, 'a', newline='') as fo:
    writer = csv.writer(fo)
    writer.writerows([[number]])

In [ ]:
list(number)

In [ ]:
with open(output_energy, 'r') as fi:
    lines = fi.readlines()

In [ ]:
test = list(map(int, lines))

In [ ]:
test = [int(x) for x[:-2] in lines]

In [ ]:
lines

In [ ]:
import string
lines[0].strip()

In [ ]:
test = list(map(int,list(map(str.strip,lines))))

In [ ]:
np.isnan(event.muon.tel[1].efficiency.width)